In [76]:
import spotipy
import spotipy.util as util
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify
from joblib import load
import pickle

In [77]:
USERNAME = 'augt12ocf9csxa4s8kbq9reg8' #your spotify username
CLIENT_ID = 'b954b92b9a674e6a9cf7322649da82d1' #set at your developer account
CLIENT_SECRET = '920cd41023994b1a99210243ce8967b2' #set at your developer account
REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

# Flask Predict function

In [ ]:
def predict(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scalar = load('scalar2.joblib')
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scalar.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = load('model2.joblib')
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')
    print('song_id_list loading...')
    song_id_list = load('song_id_list2.joblib')
    print('song_id_list loaded')
    
    print('beginning for loop...')
    for song_row in results[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flask import request
from joblib import dump
from joblib import load

import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify

# Attemping another approach using dataframe - success

In [306]:
#loop takes KNN results and filters by source track genres
# old
filtered_list = []
# filtered_lists = []
source_song_index = knn_results1[0][0]
source_genre_list = df.loc[source_song_index]["genres_stripped"].split(",")
# print(source_genre_list)
for output_song_index in knn_results1[0][1:]:
#     print(output_song_index)
    output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#     print(output_genre_list)
    for output_genre in output_genre_list:
        output_genre = output_genre.strip(' ')
#         print(output_genre)
        for source_genre in source_genre_list:
            source_genre = source_genre.strip(' ')
#             print(source_genre)
            if source_genre == output_genre:
#                 print(output_song_index,source_genre,output_genre)
                filtered_list.append(output_song_index)
#             print(filtered_list)
            else:
# #             print("source genre does not match output genre!")
                continue
#     filtered_lists.append(filtered_list)
print(len(set(filtered_list)),"!!!!!")

807 !!!!!


In [260]:
len(set(filtered_list))

376

In [308]:
knn_results1[0][0]

2

# Ready for Production - Load in Genre Array and Utilize in updated function for genre filtering

In [ ]:
def filter_model(): 
    #loop takes KNN results and filters by source track genres
    filtered_list = []
    song_list_length = 364
    source_song_index = knn_results1[0][0]
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_results1[0][1:]:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    if len(set(filtered_list)) > song_list_length:
        print("list large enough")
    else:
        counter = song_list_length - len(set(filtered_list))
        print(len(set(filtered_list)))
        print(counter)
        print(f'need to add {counter} items to final song output')
        for output_song_index in knn_results1[0][1:]:
            if output_song_index not in filtered_list:
                if counter > 0:
                    filtered_list.append(output_song_index)
                    counter -= 1
                else:
                    break
            
#     print(len(output_song_index_list),"!!!!!")    

In [ ]:
filter_model()

# Test Multiple Rows - Functional!

In [272]:
def output_len_of_lists(list_of_lists):
    for list_item in list_of_lists:
        print(len(list_item))

In [ ]:
#loop takes range of KNN results per defined track range and filter matches by source track genres
filtered_lists = []
for source_song_index in range(0,1000):
    knn_results1 = knn1.kneighbors([data_normalized[source_song_index]])[1]
    knn_result_ouputs = knn_results1[0][1:]
    filtered_list = []
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_result_ouputs:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    filtered_lists.append(set(filtered_list))
    
output_len_of_lists(filtered_lists)

# From predict.py - Production

In [5]:
def instantiate_sp(token):
    sp = spotipy.Spotify(auth=token)
    return sp

In [6]:
instantiate_sp(token)

In [7]:
def get_id(sp):
    results = sp.current_user_saved_tracks()
    song_id = results['items'][0]['track']['id']
    return song_id

In [8]:
song_id = get_id(instantiate_sp(token))

In [89]:
sp.audio

In [105]:
def get_acoustical_features(song_id,sp):
    return sp.audio_features(song_id)[0]

In [106]:
def get_popularity(trackID):
    return sp.track(trackID)['popularity']

In [107]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']

In [108]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']

In [109]:
class Sound_Drip:
    
    def __init__(self, token):
        self.token = token
    
#     def instantiate_sp(token):
#         sp = spotipy.Spotify(auth=token)
#         return sp
        
#     def instantiate_sp(token):
#         sp = spotipy.Spotify(auth=token)
#         return sp

#     def get_id(sp):
#         results = sp.current_user_saved_tracks()
#         song_id = results['items'][0]['track']['id']
#         return song_id

#     def get_acoustical_features(song_id,sp):
#         acoustical_features = sp.audio_features(song_id)[0]

#     def get_popularity(song_id):
#         popularity =  sp.track(song_id)['popularity']

#     def get_artist_id(song_id):
#         artist = sp.track(song_id)['artists'][0]['id']

#     def get_genres(artist):
#         genre = sp.artist(artist)['genres']

    
#     audio_features = {
#         "audio_features": {
#             key: results_dict[key] for key in results_dict.keys() & {
#                 'danceability',
#                 'energy',
#                 'key',
#                 'loudness',
#                 'mode',
#                 'speechiness',
#                 'acousticness',
#                 'instrumentalness',
#                 'liveness',
#                 'valence',
#                 'tempo',
#                 'time_signature'}}}

In [111]:
Sound_Drip(token)

In [113]:
dir(Sound_Drip)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [71]:
## Updated function - Not in Flask - 

def predictfunc(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
        orient='columns')
    print('Dataframe Object Created')
    print(dataframe)
    scaler = MinMaxScaler()
    print('Scaling data...')
    data_scaled = scaler.fit_transform(dataframe)
    print('Data scaled')
    normalizer = Normalizer()
    print('Normalizing data...')
    data_normalized = normalizer.fit_transform(data_scaled)
    print('Data normalized')
    print('Loading pickled model...')
    model = load('models/model2.joblib')
    print('Model loaded')
#     need to add popularity and genre filtering here
    
    results = model.kneighbors([data_scaled][0])[1]
#     genre filtering

    print('Prediction executed')
    print('song_id_list loading...')
    song_id_list = pickle.load(open('data/old_song_array.pkl', 'rb'))
    print('song_id_list loaded')

    print('beginning for loop...')
    for song_row in results[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict

In [72]:
predictfunc(content)

Loading dataframe...
Dataframe Object Created
    tempo  mode  key  loudness  instrumentalness  valence  energy  liveness  \
0  75.863     0    0    -4.647                 0    0.298   0.652     0.142   

   acousticness  speechiness  time_signature  danceability  
0         0.215       0.0406               4         0.253  
Scaling data...
Data scaled
Normalizing data...
Data normalized
Loading pickled model...


/Users/alexanderbennett/.local/share/virtualenvs/model_notebooks-FtE4iyOL/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


Model loaded
Prediction executed
song_id_list loading...
song_id_list loaded
beginning for loop...


{'songs': [{'similarity': [0.99], 'values': '41rgtFRfQKyPO6ZU40eWor'},
  {'similarity': [0.99], 'values': '1yPXig0m7bQIQ8UixN8lOJ'},
  {'similarity': [0.99], 'values': '2q2KTvUpl3KLrHMWBAmNI8'},
  {'similarity': [0.99], 'values': '5I9f40X50PyLIEvKlN1tyo'},
  {'similarity': [0.99], 'values': '3BoH4AvNZzY3cQdurkvhU4'},
  {'similarity': [0.99], 'values': '45GRVGGiLwPa2Z0BeUZcAv'},
  {'similarity': [0.99], 'values': '2m1ThHoDeAULinkfzASSqg'},
  {'similarity': [0.99], 'values': '72koDZY0TS6BmTw8kYPR7w'},
  {'similarity': [0.99], 'values': '5IOoDlyOTBGClrDRO8aZuw'},
  {'similarity': [0.99], 'values': '0Lkwk8rd0gL1XHp1QBPE6c'},
  {'similarity': [0.99], 'values': '3ytgAYdNoaywvU0ATOlrnH'},
  {'similarity': [0.99], 'values': '1jf8Uqw7jiUrzC8UjiBHgN'},
  {'similarity': [0.99], 'values': '0zU122kgLbboswDR0WCUsU'},
  {'similarity': [0.99], 'values': '4CrDyjkgbFHQrbemVWBshN'},
  {'similarity': [0.99], 'values': '09GND2Z9Gl8hOLZtL4Wlba'},
  {'similarity': [0.99], 'values': '45xTUDva3Di8eSIuAVg3uI'},

# For prediction.py - New Functions

In [ ]:
def filter_model(): 
    #loop takes KNN results and filters by source track genres
    filtered_list = []
    song_list_length = 20
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_results1[0][1:]:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    if len(set(filtered_list)) > song_list_length:
        print("list large enough")
    else:
        counter = song_list_length - len(set(filtered_list))
        print(len(set(filtered_list)))
        print(counter)
        print(f'need to add {counter} items to final song output')
        for output_song_index in knn_results1[0][1:]:
            if output_song_index not in filtered_list:
                if counter > 0:
                    filtered_list.append(output_song_index)
                    counter -= 1
                else:
                    break
            
#     print(len(output_song_index_list),"!!!!!")    

In [53]:
# import numpy as np

# old_song_list = load('data/song_id_list2(1).joblib')

# old_song_array = np.array(old_song_list)

# pickle.dump(old_song_array, open('data/old_song_array.pkl', 'wb'))